# 2.5 自动微分
- **目录**
  - 2.5.1 一个简单的例子
  - 2.5.2 非标量变量的反向传播
  - 2.5.3 分离计算
  - 2.5.4 Python控制流的梯度计算

- 求导是几乎所有深度学习优化算法的关键步骤。
- 深度学习框架通过自动计算导数，即**自动微分（automatic differentiation）** 来加快求导。
- 在深度学习中，尤其在Pytorch等框架中，系统会根据用户设计的模型构建一个**计算图（computational graph）**，用于跟踪计算是哪些数据通过哪些操作组合起来并产生输出。

- **反向传播（backpropagate）** 意味着跟踪整个计算图，填充关于每个参数的偏导数。

---------
- **说明：何为计算图？**
  - 计算图是一个有向无环图（DAG, Directed Acyclic Graph），用于表示计算过程。
    - 图中的节点表示操作（如加法、乘法）或变量（如张量、权重），边表示操作的输入和输出关系。
    - 在深度学习和自动微分框架中，计算图是理解模型计算以及梯度计算的核心概念。
  - 计算图的基本概念：
    - **节点（Nodes）**：
      - **操作节点（Operation Nodes）**：表示各种数学操作，如加法、乘法、矩阵乘法、激活函数等。
      - **变量节点（Variable Nodes）**：表示输入数据、模型参数、常数等。
    - **边（Edges）**：
      - 表示数据在节点之间的流动，即操作的输入和输出关系。
  - 计算图的构建过程：
    - **定义输入**：
      - 将输入变量定义为计算图中的节点。例如，输入数据和模型参数。
    - **定义操作**：
      - 通过对输入变量进行各种操作（如加法、乘法等），构建计算图。这些操作节点会连接到相应的输入变量节点上。
    - **计算输出**：
      - 最终的输出节点通过一系列操作节点和输入节点相连，形成一个完整的计算图。
  - 计算图的功能
    - **前向传播（Forward Pass）**：
      - 通过计算图从输入节点开始依次计算每个操作节点，直到得到最终的输出。前向传播用于计算模型的预测结果或损失。
    - **反向传播（Backward Pass）**：
      - 通过计算图从输出节点开始依次向前计算每个操作节点的梯度，直到得到输入节点的梯度。
      - 反向传播用于计算模型参数的梯度，以便进行梯度下降优化。

  - 计算图的类型
    - **静态计算图（Static Computation Graph）**：
      - 在模型运行之前，计算图是固定的，不能动态改变。例如，TensorFlow的静态图模式。
    - **动态计算图（Dynamic Computation Graph）**：
      - 计算图在每次前向传播时动态生成，可以根据需要进行修改。例如，PyTorch和TensorFlow的Eager Execution模式。

  - 一个简单的计算图示例，用于计算以下表达式的前向传播和反向传播过程：
    $$ z = (x + y) \times w $$
  - 前向传播：
    - **定义输入**：$x$, $y$, $w$ 为输入变量节点。
    - **定义操作**：
      - 加法操作：$a = x + y$
      - 乘法操作：$z = a \times w$
  - 计算图如下：
```
  x   y  w
   \ /   |
    +    |
   / \   |
  a   \  |
       \ |
        * 
        |
        z
```
  - 反向传播，计算每个变量的梯度：
    - **输出节点的梯度**：
      - $\frac{\partial z}{\partial z} = 1$
    - **乘法节点的梯度**：
      - $\frac{\partial z}{\partial a} = w$
      - $\frac{\partial z}{\partial w} = a$
    - **加法节点的梯度**：
      - $\frac{\partial a}{\partial x} = 1$
      - $\frac{\partial a}{\partial y} = 1$
    - 通过链式法则，计算每个输入变量的梯度：
      - $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial a} \cdot \frac{\partial a}{\partial x} = w \cdot 1 = w$
      -  $\frac{\partial z}{\partial y} = \frac{\partial z}{\partial a} \cdot \frac{\partial a}{\partial y} = w \cdot 1 = w$
      - $\frac{\partial z}{\partial w} = a = x + y$

-------------------


## 2.5.1 一个简单的例子

- **对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**。

In [1]:
import torch
#创建变量`x`并为其分配一个初始值。
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

- 深度学习一般会大规模更新相同的参数（GPT-3模型参数规模达1750亿）。
- 每次都分配新的内存可能很快就会将内存耗尽。
- 因此深度学习参数更新（尤其是权重更新）一般采用**就地操作（in-place）**。
- 注意，一个**标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状**。

In [2]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None,即梯度为空

- 在Tensor上调用`backward()`计算导数。
- 如果 Tensor 是一个标量(即它包含一个元素的数据），则不需要为` backward() `指定任何参数。
- 如果该Tensor有多个元素(比如向量或矩阵)，则需要指定一个`gradient` 参数，该参数是形状匹配的张量。

In [3]:
# 计算y
y = 2 * torch.dot(x, x)
# 此时的y是一个标量，即只有一个值
y 

tensor(28., grad_fn=<MulBackward0>)

- `x`是一个长度为4的向量，计算`x`和`x`的点积得到的`y`是一个的标量。
- 然后**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**，并打印这些梯度。


In [4]:
# 反向传播函数自动计算梯度，由于此时的y是一个标量，因此可以计算梯度
y.backward()

In [5]:
# 即y=2x^2,那么y'=4x，那么此时y关于x每个分量的梯度为4*[0,1,2,3]
x.grad

tensor([ 0.,  4.,  8., 12.])

- 函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于$\mathbf{x}$的梯度应为$4\mathbf{x}$。
- 根据此公式验证手工计算的梯度和上述反向传播计算后的梯度是否一致。


In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

- 计算`x`的另一个函数，然后计算梯度


In [8]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_() #清除梯度
y = x.sum()
y.backward()
x.grad,type(y), y # 此时的y也是一个标量，但是为何这么操作?

(tensor([1., 1., 1., 1.]), torch.Tensor, tensor(6., grad_fn=<SumBackward0>))

In [9]:
#清除梯度
x.grad.zero_()
y=x**2
##为何求和？主要是为了使y成为一个变量。可以参考这个帖子：https://blog.csdn.net/qq_39208832/article/details/117415229
#解释更透彻的是这个帖子：https://zhuanlan.zhihu.com/p/27808095
y.sum().backward()
x.grad,y #仍是前面那个求和后的y

(tensor([0., 2., 4., 6.]), tensor([0., 1., 4., 9.], grad_fn=<PowBackward0>))

-------

- **说明**：在对函数求梯度时，**是否要求因变量是标量**？
  - 在对函数求梯度时，**通常要求因变量是标量**。
    - 梯度的定义和计算涉及到一个标量值函数（单一输出）关于其自变量的偏导数。
    - 当因变量是标量时，可以计算该函数**在某个点处关于各个自变量分量的偏导数，并将它们组合成梯度向量**。
  - 如果因变量不是标量（即函数有多个输出），那么**通常会针对每个输出分别计算梯度**。
    - 在这种情况下，可以使用雅可比矩阵来表示所有输出关于自变量的梯度信息。
    - 雅可比矩阵的第$i$ 行、第 $j$ 列元素表示第 $i$ 个输出关于第 $j$ 个自变量分量的偏导数：
      $$J[i, j] = ∂f_i/∂x_j$$
      其中 $f_i$ 是第 $i$ 个输出，$x_j$ 是第 $j$ 个自变量分量。
  - 总结一下：在对函数求梯度时，通常要求因变量是标量。如果函数有多个输出，可以针对每个输出分别计算梯度，并使用雅可比矩阵表示所有梯度信息。

In [10]:
import torch

def f(u, v):
    y = torch.zeros(2)
    y[0] = u[0]**2 + u[1]*v[0]
    y[1] = u[1]*v[1] + v[0]*v[1]**2
    return y

# 创建自变量张量
u = torch.tensor([1.0, 2.0], requires_grad=True)
v = torch.tensor([3.0, 4.0], requires_grad=True)

# 计算输出向量，y本质上就是有个两个函数组成的向量而已
# 计算梯度时分别针对y中的每个函数y[0],y[1]计算关于u,v的梯度
y = f(u, v)

# 初始化雅可比矩阵
jacobian = torch.zeros(2, 4)

# 计算雅可比矩阵的每个元素
for i in range(2):
    # 对 y[i] 进行反向传播以计算梯度
    y[i].backward(retain_graph=True)
    
    # 提取梯度并存储在雅可比矩阵中
    jacobian[i, :2] = u.grad
    jacobian[i, 2:] = v.grad

    # 清零梯度，以便进行下一次迭代
    u.grad.zero_()
    v.grad.zero_()

print("Jacobian matrix:")
print(jacobian)


Jacobian matrix:
tensor([[ 2.,  3.,  2.,  0.],
        [ 0.,  4., 16., 26.]])


- 上述代码的输出结果可解释为：
$$
\left[
\begin{array}{cccc} 
    ∂y[0]/∂u[0] = 2  & ∂y[0]/∂u[1] = 3  & ∂y[0]/∂v[0] = 2 &  ∂y[0]/∂v[1] = 0\\ 
    ∂y[1]/∂u[0] = 0  &  ∂y[1]/∂u[1] = 4   & ∂y[1]/∂v[0] = 16 & ∂y[1]/∂v[1] = 26\\ 
\end{array}
\right]
$$

-----------

## 2.5.2 非标量变量的反向传播

- 当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个矩阵，即**雅克比矩阵**。
- 对于高阶和高维的`y`和`x`，求导的结果可以是一个<b>高阶张量。
- 然而，虽然这些更奇特的对象(即作为求导结果的高阶张量)确实出现在高级机器学习中（包括深度学习中），但当调用向量的反向传播计算时，通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
- **这样做的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和**。
- 对上述这句话以及下述代码注释的理解：
  - 原代码中这段注释不好理解，而且例子也没有体现出为什么要求偏导数的和，因为此处的x只有一条样本数据。
如果x是类似[[0, 1, 2, 3],[6, 7, 8, 9]]这样的数据，这样小批量样本中就有2条数据。
  - 原注释中的"只想求偏导数的和"是指：
    - 分别针对[0, 1, 2, 3]和[6, 7, 8, 9]两条数据计算偏导数，得出两个梯度向量：
    - [0., 2., 4., 6.]和[12, 14, 16, 18]，然后再将二者按元素相加求和得到[12， 16， 20， 24]。
    - 最后将这个向量作为某个小批量样本数据的总体梯度。
    - 再将这个梯度作为后续工作的基础，比如用于更新权重等参数。
  - 这种做法就是后面内容中的优化算法——小批量随机梯度下降的做法。
  - 所谓“所以传递一个1的梯度是合适的.”这句话是指将梯度权重因子设为1，即将梯度乘以权重1，也就是不改变梯度的计算结果。

In [11]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的.
x.grad.zero_()
y = x * x
# 注意此处没有调用y.sum(),那么需指定backward函数的gradient参数（即梯度权重因子）的值
# 该参数形状与x相同
y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

In [12]:
# y的值
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [13]:
# 和上述代码的计算结果是一致的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad, y

(tensor([0., 2., 4., 6.]), tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>))

--------

- **说明**：`y.sum().backward()`与`y.backward(torch.ones(len(x)))`区别与联系
  - $\mathbf x$ 向量可以看成向量$[x_1, x_2, x_3, x_4]$，分别对应$[0., 1., 2., 3.]$，`y.sum()`对应$y = x_1^2 + x_2^2 + x_3^2 + x_4^2$，然后调用`backward()`计算梯度，本质上就是针对$y$分别计算关于$x_1, x_2, x_3, x_4$的偏导数，即$[2x_1,2x_2, 2x_3, 2x_4]$。代入$[0., 1., 2., 3.]$，偏导数值为：`[0., 2., 4., 6.]`
  - `y.backward(torch.ones(len(x)))`函数如果不指定`gradient`参数则会报错。如果指定该参数比如此处的`torch.ones(len(x))`即`[1, 1, 1, 1]`，本质上是一个权重因子，用于调整每个$\mathbf x$中每个分量的梯度值，即在计算了`y`关于每个分量的梯度之后，需要将梯度与对应的权重因子相乘，然后作为梯度保存。

- **示例：**
  - `y.mean().backward()`：`y.mean()`对应$y = {\frac14}{(x_1^2 + x_2^2 + x_3^2 + x_4^2)}$，然后调用`backward()`计算梯度，本质上就是针对$y$分别计算关于$x_1, x_2, x_3, x_4$的偏导数，即$[ {\frac12}x_1, {\frac12}x_2,  {\frac12}x_3,  {\frac12}x_4]$。代入$[0., 1., 2., 3.]$，那么偏导数值即为：`[0., 0.5, 1.,1.5]`

In [14]:
x.grad.zero_()
y = x * x
y.mean().backward()
x.grad

tensor([0.0000, 0.5000, 1.0000, 1.5000])

- **示例：**
  - `y.backward`指定`gradient`参数的值

In [15]:
x.grad.zero_()
y = x * x
# 设定x每个分量的梯度权重因子值为3
y.backward(gradient=torch.tensor([3, 3, 3, 3]))
x.grad

tensor([ 0.,  6., 12., 18.])

- 注意：
  - 再次强调，在`backward()`函数的上下文中，`gradient` 参数实际上是梯度因子，而不是梯度本身，在此处作为权重因子参与梯度计算。

## 2.5.3 分离计算

- 所谓**分离计算**就是将某些计算移动到计算图之外，也就是在计算图中的某处**断开自动求导的传播**。
- 例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。想计算`z`关于`x`的梯度，但由于某种原因希望将`y`视为一个常数，并且只考虑到`x`在`y`被计算后发挥的作用。
- 此时，可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，但丢弃计算图中如何计算`y`的任何信息。
- 换句话说，梯度不会向后流经`u`到`x`。
- 下例就是利用反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，而不是`z=x*x*x`关于`x`的偏导数。


In [16]:
x.grad.zero_() # 梯度清零
y = x * x # y = x^2

## 调用detach，将y剥离梯度计算，只是将其算术计算的结果赋给y
# 也就是x*x的计算结果，此处u等于[0,1,2,3]*[0,1,2,3] = [0,1,4,9]
u = y.detach() 
z = u * x # 如果对其求梯度，x的梯度即u

z.sum().backward() #此处只求x的梯度，梯度就是u的值[0,1,4,9]
x.grad == u,u,x.grad

(tensor([True, True, True, True]),
 tensor([0., 1., 4., 9.]),
 tensor([0., 1., 4., 9.]))

## 2.5.4 Python控制流的梯度计算

- **即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），仍然可以计算得到的变量的梯度**。
- **示例：**
  - 在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

- 计算梯度


In [18]:
# a是一个随机数，因此每次运行下列代码，a的值及其梯度可能不同
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad, a, d, d/a

(tensor(2048.),
 tensor(0.8392, requires_grad=True),
 tensor(1718.6700, grad_fn=<MulBackward0>),
 tensor(2048., grad_fn=<DivBackward0>))

In [19]:
# 标量的范数
t = torch.tensor(-5.0)
t,t.norm() # 标量的范数等于其绝对值

(tensor(-5.), tensor(5.))

- 上述代码中梯度的计算过程：
  - 调用 `d.backward()`。这将触发关于 a 的梯度计算。根据链式法则，需要计算 $dc/da$。由于`c` 的值取决于 `b` 和条件语句，因此我们需要根据不同情况来计算梯度。
  - 如果 `b.sum() > 0` 成立，那么 `c = b`，并且 $dc/db = 1$。在这种情况下，需要计算 $db/da$。由于 `b` 是通过重复乘以 `2` 计算的，可以表示为 $b = a * 2^n$（其中 $n$ 是满足`b.norm() >= 1000` 的最小整数）。因此，$db/da = 2^n$。最后，根据链式法则，$dc/da = dc/db * db/da = 1 * 2^n = 2^n$。
  - 如果`b.sum() <= 0`成立，那么`c = 100 * b`，并且 $dc/db = 100$。在这种情况下，还是需要计算$db/da$，它的值仍然是 $2^n$。
  - 最后，根据链式法则，$dc/da = dc/db * db/da = 100 * 2^n$。
  - 计算完成后，`a.grad` 存储了关于`a` 的梯度值。

- 对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a，因此可以用d/a验证梯度是否正确。
  - 也就是说：不管是迭代还是if分支流程的控制，函数f(a)本质上就是关于a的乘法运算，即d=f(a)=k*a，那么其梯度就是k，即等于d/a。

In [20]:
a.grad == d / a

tensor(True)

-  计算z = (x + y)* w表达式的前向传播和反向传播过程

In [21]:
# 计算z = (x + y)* w的计算图
import torch

# 定义输入变量，并设置requires_grad=True以便计算梯度
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)
w = torch.tensor(4.0, requires_grad=True)

# 前向传播
a = x + y
z = a * w

# 打印前向传播的结果
print(f"a = x + y: {a.item()}")
print(f"z = a * w: {z.item()}")

# 反向传播，计算梯度
z.backward()

# 打印各个变量的梯度
print(f"dz/dx: {x.grad.item()}")
print(f"dz/dy: {y.grad.item()}")
print(f"dz/dw: {w.grad.item()}")

a = x + y: 5.0
z = a * w: 20.0
dz/dx: 4.0
dz/dy: 4.0
dz/dw: 5.0


## 小结

* 深度学习框架可以自动计算导数：首先将梯度附加到想要对其计算偏导数的变量上。
* 然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

